In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
directory = '../input/plant-pathology-2021-fgvc8/train_images/'
directory2 = '../input/plant-pathology-2021-fgvc8/test_images/'

In [ ]:
trainfile=[]
for file in os.listdir(directory):
    trainfile+=[file]
print(len(trainfile))

testfile=[]
for file in os.listdir(directory2):
    testfile+=[file]
print(len(testfile))

In [ ]:
print(trainfile[0:3])

In [ ]:
train=pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
train

In [ ]:
Name=train['labels'].unique()
print(train['labels'].unique())
print(train['labels'].nunique())

In [ ]:
N=[]
for i in range(len(Name)):
    N+=[i]
    
mapping=dict(zip(Name,N)) 
reverse_mapping=dict(zip(N,Name)) 

def mapper(value):
    return reverse_mapping[value]

In [ ]:
train_imdata=[]
train_imfile=[]

for im in tqdm(os.listdir(directory)):
    image=load_img(os.path.join(directory,im), grayscale=False, color_mode='rgb', target_size=(60,60))
    image=img_to_array(image)
    image=image/255.0
    train_imdata.append(image)
    train_imfile.append(im)

In [ ]:
test_imdata=[]
test_imfile=[]

for im in tqdm(os.listdir(directory2)):
    image=load_img(os.path.join(directory2,im), grayscale=False, color_mode='rgb', target_size=(60,60))
    image=img_to_array(image)
    image=image/255.0
    test_imdata.append(image)
    test_imfile.append(im)

In [ ]:
label_num=[]
for item in train['labels']:
    value=mapping[item]
    label_num+=[value]
train['label_num']=label_num
train

In [ ]:
data=np.array(train_imdata)
print(data.shape)

In [ ]:
test=np.array(test_imdata)
print(test.shape)

In [ ]:
label0=[]
for item in train_imfile:
    label0+=[train[train['image']==item].iloc[0,2]]
#print(label0)

labels1=to_categorical(label0)
labels=np.array(labels1)
#print(labels.shape)

In [ ]:
trainx,testx,trainy,testy=train_test_split(data,labels,test_size=0.2,random_state=44)

In [ ]:
print(trainx.shape)
print(testx.shape)
print(trainy.shape)
print(testy.shape)

In [ ]:
datagen = ImageDataGenerator(horizontal_flip=True,vertical_flip=True,rotation_range=20,zoom_range=0.6,
                        width_shift_range=0.2,height_shift_range=0.2,shear_range=0.1,fill_mode="nearest")

In [ ]:
densenet_weights_path = '../input/densenet201/densenet201_notop.h5'

In [ ]:
pretrained_model3 = tf.keras.applications.DenseNet201(input_shape=(60,60,3),include_top=False,weights=densenet_weights_path,pooling='avg')
pretrained_model3.trainable = False

In [ ]:
inputs3 = pretrained_model3.input
x3 = tf.keras.layers.Dense(128, activation='relu')(pretrained_model3.output)
outputs3 = tf.keras.layers.Dense(12, activation='softmax')(x3)
model = tf.keras.Model(inputs=inputs3, outputs=outputs3)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
his=model.fit(datagen.flow(trainx,trainy,batch_size=16),validation_data=(testx,testy),epochs=30)

In [ ]:
y_pred=model.predict(testx)
pred=np.argmax(y_pred,axis=1)
ground = np.argmax(testy,axis=1)
print(classification_report(ground,pred))

In [ ]:
get_acc = his.history['accuracy']
value_acc = his.history['val_accuracy']
get_loss = his.history['loss']
validation_loss = his.history['val_loss']

epochs = range(len(get_acc))
plt.plot(epochs, get_acc, 'r', label='Accuracy of Training data')
plt.plot(epochs, value_acc, 'b', label='Accuracy of Validation data')
plt.title('Training vs validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
epochs = range(len(get_loss))
plt.plot(epochs, get_loss, 'r', label='Loss of Training data')
plt.plot(epochs, validation_loss, 'b', label='Loss of Validation data')
plt.title('Training vs validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
load_img("../input/plant-pathology-2021-fgvc8/test_images/85f8cb619c66b863.jpg",target_size=(60,60))

In [ ]:
image=load_img("../input/plant-pathology-2021-fgvc8/test_images/85f8cb619c66b863.jpg",target_size=(60,60))

image=img_to_array(image) 
image=image/255.0
prediction_image=np.array(image)
prediction_image= np.expand_dims(image, axis=0)

In [ ]:
prediction=model.predict(prediction_image)
value=np.argmax(prediction)
move_name=mapper(value)
#print(prediction)
#print(value)
print("Prediction is {}.".format(move_name))

In [ ]:
print(test.shape)
prediction2=model.predict(test)
print(prediction2.shape)

PRED=[]
for item in prediction2:
    value2=np.argmax(item)
    name2=mapper(value2)
    PRED+=[name2]

In [ ]:
sample=pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
sample

In [ ]:
sample['image']=test_imfile
sample['labels']=PRED
subm=sample.sort_values('image').reset_index(drop=True)
subm.to_csv('submission.csv', index=False)
subm